In [17]:
# import 필요한 라이브러리
import os
from natsort import natsorted
from pathlib import Path
import shutil
from os import listdir
from os.path import isdir
from os.path import exists
from os.path import isfile, join
import numpy as np
from tqdm.notebook import tqdm
import re
import pandas as pd
import datetime
import random
from natsort import natsorted
import secrets

In [18]:
## 질문1. 사실상 이 코드 필요없나요?
# 필요한 함수 정의
def ensure_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)


class Generator:
    def __init__(self, start_date, end_date, start_id, end_id):
        self.start_date = datetime.date(
            start_date["year"], start_date["month"], start_date["day"]
        )
        self.end_date = datetime.date(
            end_date["year"], end_date["month"], end_date["day"]
        )
        self.start_id = start_id
        self.end_id = end_id

    def dategenerate(self, qty):
        start_date = self.start_date
        end_date = self.end_date

        set_date = set()
        while len(set_date) < qty:
            time_between_dates = end_date - start_date
            days_between_dates = time_between_dates.days
            random_number_of_days = random.randrange(days_between_dates)
            random_date = start_date + datetime.timedelta(days=random_number_of_days)
            gen_date = "".join(str(random_date).split("-"))
            set_date.add(gen_date)
        return natsorted(list(set_date))

    def idgenerate(self, isconstlength=False, maxlength=3):
        set_id = set()
        for i in range(self.start_id, self.end_id):
            str_id = str(i)
            if isconstlength:
                str_id = "0" * (maxlength - len(str_id)) + str_id
            gen_id = "W" + str_id
            set_id.add(gen_id)

        return natsorted(list(set_id))


def makeColumns():
    sideType = [
        "Left",
        "Right",
    ]
    sensorType = ["ACC", "Gyro", "Mag"]
    AxisType = ["X", "Y", "Z"]
    lside = len(sideType)
    lsen = len(sensorType)
    lAxis = len(AxisType)
    col_acc = [
        f"{sideType[i//(lsen*lAxis)]}_{sensorType[(i//(lsen))%lsen]}_{AxisType[i%lAxis]}"
        for i in range(0, lsen * lAxis * lside)
    ]
    return col_acc


def genQST():
    list_Test = ["Walking", "Running", "Sitting", "Sleeping"]
    list_Location = ["Boston", "NYC", "Busan", "Seoul"]
    list_Comments = ["", "Need_discussion", "Serious_Problem"]

    data = []
    for test in list_Test:
        for _ in range(0,5):
            data.append([test,secrets.choice(list_Location),random.random(),secrets.choice(list_Comments)])
    return data



In [19]:
# 실험기간
start_date = {"year": 2022, "month": 3, "day": 1}
end_date = {"year": 2022, "month": 6, "day": 1}
# 환자 ID
start_id = 1
end_id = 21
date_exp = Generator(start_date, end_date, start_id, end_id)


In [20]:
list_patient = date_exp.idgenerate(True)
targetFolder = "Data_QST_IMU"
data_length = 100
ensure_dir(targetFolder)
for patient in list_patient:
    ensure_dir(join(targetFolder, patient))
    # 방문일 뽑기
    visitCount = random.randrange(6)
    list_visitDate = date_exp.dategenerate(visitCount)
    # 방문 안하고 도망갔을 경우, 실험 폴더가 없음
    if len(list_visitDate) == 0:
        continue
    # 방문날짜가 정해졌을 경우, 이를 폴더로 만들기
    for visitDate in list_visitDate:
        # QST 저장
        saveDir = join(targetFolder, patient, visitDate, "QST")
        ensure_dir(saveDir)
        data_qst = genQST()
        result_qst = pd.DataFrame(data_qst,columns=["Test", "Location", "Value", "Comments"])
        result_qst.to_excel(join(saveDir, f"{patient}_{visitDate}_QST.xlsx"))
        # IMU 저장
        saveDir = join(targetFolder, patient, visitDate, "IMU")
        ensure_dir(saveDir)
        list_cols = makeColumns()
        data_subject = np.random.rand(data_length, len(list_cols))
        result_qst = pd.DataFrame(data_subject,columns=list_cols)
        result_qst.to_excel(join(saveDir, f"{patient}_{visitDate}_IMU.xlsx"))